In [1]:
import pandas as pd
import numpy as np
import datetime
import math as m
from sklearn.impute import KNNImputer,SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import re
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',200)

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia.pkl")

In [ ]:
df.columns

In [ ]:
df.head()

# Sin seccionar el df

In [ ]:
sc=StandardScaler()

In [ ]:
continuas=['Temperature(F)', 'Humidity(%)','Pressure(in)', 'Visibility(mi)'
           ,'Wind_Speed(mph)']

In [ ]:
sc.fit(df[continuas])

In [ ]:
X=sc.transform(df[continuas])

In [ ]:
X=pd.DataFrame(X,columns=df[continuas].columns)

In [ ]:
X.columns = [str(col) + '_Escalado' for col in X.columns]

In [ ]:
df.shape

In [ ]:
df=pd.concat([df,X],axis=1)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

# Imputacion de variables

In [ ]:
def get_value_off_missing_data(data_original,data_impute):
#     f = plt.subplots()    
    sns.distplot(data_original.dropna(),label="datos originales",hist=False,)
    sns.distplot(data_impute,color="red",label="datos imputados",hist=False,)
    plt.legend()

In [ ]:
imputar_mult=['Temperature(F)_Escalado','Humidity(%)_Escalado', 'Pressure(in)_Escalado',
       'Visibility(mi)_Escalado', 'Wind_Speed(mph)_Escalado']

In [ ]:
imputador = KNNImputer(n_neighbors=7)

In [ ]:
imputador.fit(df[imputar_mult])

In [ ]:
data_knn_imputer=pd.DataFrame(imputador.transform(df[imputar_mult]),columns=df[imputar_mult].columns)

In [ ]:
data_knn_imputer.columns = [str(col) + '_KNN' for col in data_knn_imputer.columns]

In [ ]:
df1=pd.concat([df,data_knn_imputer],axis=1)

##### imputación por moda

In [ ]:
df1["Side"].value_counts(dropna=False)

In [ ]:
df1["Side_moda"] = df1["Side"].fillna("R")

In [ ]:
df_side = df1.groupby('Side_moda').size()
cols=["red","blue"]
cases=["Izquierda","Derecha"]
plt.figure(figsize=(5,5))
plt.pie(df_side,
        colors = cols,
        labels= cases,
        explode = (0,0.1), #moving slices apart
        autopct = ('%1.1f%%')) #to display %
plt.title('¿De que lado ocurren los accidentes?', weight='heavy', fontsize=8, style='italic');

In [ ]:
df1["Wind_Direction"].value_counts(dropna=False)

In [ ]:
df1["Wind_Direction_moda"] = df1["Wind_Direction"].fillna("SW")

In [ ]:
df1["Wind_Direction_moda"].value_counts(dropna=False)

In [ ]:
df1["Weather"].value_counts(dropna=False)

In [ ]:
df1["Weather_moda"] = df1["Weather"].fillna("Cloudy")

In [ ]:
df1["Weather_moda"].value_counts(dropna=False)

### guardamos las escalas y el modelo

In [ ]:
import pickle

* pickle.dump(scaler,open("scaler.pkl","wb"))##write binary
* scaler=pickle.load(open("scaler.pkl","rb"))##read binary

In [ ]:
#pickle.dump(sc,open("scaler_forma1.pkl","wb"))
#pickle.dump(imputador,open("imputador_forma1.pkl","wb"))

In [ ]:
#pd.to_pickle(df1,"Tabla_Proyecto_Limpia_forma1.pkl")

In [2]:
df1=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
df1.columns

In [ ]:
df1.isnull().sum()

In [ ]:
sns.countplot(df1["Wind_Direction"],
             order = df1["Wind_Direction"].value_counts().index)
sns.despine()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(df1["Weather"],
             order = df1["Weather"].value_counts().index)
sns.despine()

In [ ]:
df1.loc[df1["Weather"]=="Cloudy",["Weather","Weather_Condition"]].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(df1.loc[df1["Weather"]=="Cloudy","Weather_Condition"],
             order = df1.loc[df1["Weather"]=="Cloudy","Weather_Condition"].value_counts().index)
sns.despine()

# Proseguimos

In [ ]:
get_value_off_missing_data(df1['Humidity(%)_Escalado'],df1['Humidity(%)_Escalado_KNN'])

In [ ]:
get_value_off_missing_data(df1['Pressure(in)_Escalado'],df1['Pressure(in)_Escalado_KNN'])

In [ ]:
get_value_off_missing_data(df1['Temperature(F)_Escalado'],df1['Temperature(F)_Escalado_KNN'])

# Componentes Principales

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_components=3

In [ ]:
pca = PCA(n_components=n_components)

In [ ]:
c=['Temperature(F)_Escalado_KNN','Humidity(%)_Escalado_KNN', 'Pressure(in)_Escalado_KNN',
       'Visibility(mi)_Escalado_KNN', 'Wind_Speed(mph)_Escalado_KNN']

In [ ]:
Xpca = pd.DataFrame(pca.fit_transform(df1[c]),columns=['z%d'%(i+1) for i in range(0,n_components)])

In [ ]:
Xpca

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
Xpca=pd.concat([df1["Severity"],Xpca],axis=1)

In [ ]:
Xpca

In [ ]:
sns.lmplot(data=Xpca[['z1','z2',"Severity"]],x='z1',y='z2',fit_reg=False,hue="Severity")
sns.despine()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xpca['z1'],Xpca['z2'],Xpca['z3'] , c=Xpca['Severity'] )
sns.despine()

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
n_components = 3

In [ ]:
lda = LDA(n_components=3)

In [ ]:
#Xlda=lda.fit_transform(df1[c],df1["Severity"])

In [ ]:
Xlda=lda.fit(df1[c],df1["Severity"])

In [ ]:
Xlda.explained_variance_ratio_.cumsum()

In [ ]:
Xlda=lda.fit_transform(df1[c],df1["Severity"])

In [ ]:
Xlda = pd.DataFrame(Xlda,columns=['z%d'%(i+1) for i in range(0,n_components)])

In [ ]:
Xlda=pd.concat([df1["Severity"],Xlda],axis=1)

In [ ]:
sns.lmplot(data=Xlda[['z1','z2',"Severity"]],x='z1',y='z2',fit_reg=False,hue="Severity")
sns.despine()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xlda['z1'],Xlda['z2'],Xlda['z3'] , c=Xlda['Severity'] )
sns.despine()

# Transformación entrópica

In [3]:
def iv(target,df,var):
    target =target
    iv_dict = {}
    for f in var:
        aux = df[[f,target]].copy()
        aux['n'] = 1

        aux = aux.pivot_table(columns=target,
                              index=f,
                              values='n',
                              aggfunc='count',
                              fill_value=0)

        aux['pne'] = aux[0]/aux[0].sum()
        aux['pe'] = aux[1]/aux[1].sum()

        aux['woe'] = np.log(aux['pne']/aux['pe'])

        aux['iv'] = aux['woe']*(aux['pne']-aux['pe'])

        iv_dict[f]=aux['iv'].sum()
    return iv_dict

In [4]:
 df1.columns

Index(['Source', 'TMC', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'Distance(mi)', 'Description', 'Street', 'Side', 'City',
       'County', 'State', 'Zipcode', 'Timezone', 'Airport_Code',
       'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition', 'Amenity',
       'Crossing', 'Junction', 'Station', 'Stop', 'Traffic_Signal',
       'TimeMinutes', 'Hour', 'Weekday', 'Year', 'Month', 'Day', 'County_Freq',
       'State_Freq', 'City_Freq', 'Airport_Freq', 'Weather',
       'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado', 'Temperature(F)_Escalado_KNN',
       'Humidity(%)_Escalado_KNN', 'Pressure(in)_Escalado_KNN',
       'Visibility(mi)_Escalado_KNN', 'Wind_Speed(mph)_Escalado_KNN',
       'Side_moda', 'Wind_Direction_moda', 'Weather_moda'],
      dtype='object')

In [ ]:
 df1['Distance(mi)_cut']=pd.cut(df1['Distance(mi)'],bins=6).astype(str)

In [ ]:
 df1['Temperature(F)_Escalado_KNN_cut']=pd.cut(df1['Temperature(F)_Escalado_KNN'],bins=6).astype(str)

In [ ]:
df1['Humidity(%)_Escalado_KNN_cut']=pd.cut(df1['Humidity(%)_Escalado_KNN'],bins=6).astype(str)

In [ ]:
df1['Pressure(in)_Escalado_KNN_cut']=pd.cut(df1['Pressure(in)_Escalado_KNN'],bins=6).astype(str)

In [5]:
cat=['State','Timezone','Wind_Direction', 'Amenity','Crossing', 'Junction', 'Station', 'Stop', 
     'Traffic_Signal','Hour', 'Weekday', 'Year', 'Month', 'Day', 'County_Freq',
       'State_Freq', 'City_Freq', 'Airport_Freq', 'Weather','Side_moda', 'Wind_Direction_moda', 'Weather_moda']

In [6]:
cols_interes=cat##+[col for col in df1.columns if col.endswith('_cut')]

In [7]:
cols_interes

['State',
 'Timezone',
 'Wind_Direction',
 'Amenity',
 'Crossing',
 'Junction',
 'Station',
 'Stop',
 'Traffic_Signal',
 'Hour',
 'Weekday',
 'Year',
 'Month',
 'Day',
 'County_Freq',
 'State_Freq',
 'City_Freq',
 'Airport_Freq',
 'Weather',
 'Side_moda',
 'Wind_Direction_moda',
 'Weather_moda']

### generamos el target

In [8]:
dic={4:"fuerte",3:"fuerte",2:"debil",1:"debil"}

In [9]:
df1["tgt"]=df1["Severity"].replace(dic)

In [10]:
dic={"fuerte":1,"debil":0}

In [11]:
df1["tgt"]=df1["tgt"].replace(dic)

In [12]:
df1["tgt"].value_counts(1)

0    0.689953
1    0.310047
Name: tgt, dtype: float64

In [13]:
dic_iv=iv(target="tgt",df=df1,var=cols_interes)

In [14]:
criterio_iv=pd.DataFrame(dic_iv,index=[0]).T.reset_index().rename(columns={'index':'variable',0:'iv'})

In [15]:
criterio_iv=criterio_iv.sort_values(by=["iv"],ascending=False)

In [16]:
criterio_iv=criterio_iv.reset_index(drop=True)

In [17]:
criterio_iv

,variable,iv
0,Airport_Freq,0.812268
1,City_Freq,0.757369
2,County_Freq,0.693243
3,Side_moda,0.274326
4,State,0.250359
5,State_Freq,0.250359
6,Traffic_Signal,0.240896
7,Crossing,0.132997
8,Weekday,0.052965
9,Year,0.045739


In [18]:
criterio_iv[(criterio_iv["iv"]>.5)|(criterio_iv["iv"]<.02)]

,variable,iv
0,Airport_Freq,0.812268
1,City_Freq,0.757369
2,County_Freq,0.693243
15,Wind_Direction,0.018076
16,Wind_Direction_moda,0.018076
17,Junction,0.017200
18,Amenity,0.016615
19,Station,0.016284
20,Month,0.005865
21,Day,0.000713


In [19]:
var_iv=["Airport_Freq","City_Freq","County_Freq","Wind_Direction","Wind_Direction_moda",
        "Junction","Amenity","Station","Month","Day"]

In [20]:
df1=df1.drop(var_iv,axis=1)

In [23]:
test=['Weather_Condition','Side','Weather','State','Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Speed(mph)','Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado','Wind_Speed(mph)_Escalado',
     'tgt']

In [24]:
df1=df1.drop(test,axis=1)

In [25]:
df1.columns

Index(['Source', 'TMC', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County',
       'Zipcode', 'Timezone', 'Airport_Code', 'Crossing', 'Stop',
       'Traffic_Signal', 'TimeMinutes', 'Hour', 'Weekday', 'Year',
       'State_Freq', 'Temperature(F)_Escalado_KNN', 'Humidity(%)_Escalado_KNN',
       'Pressure(in)_Escalado_KNN', 'Visibility(mi)_Escalado_KNN',
       'Wind_Speed(mph)_Escalado_KNN', 'Side_moda', 'Weather_moda'],
      dtype='object')

In [27]:
df1.shape

(2984488, 30)

# Kbest

In [38]:
df1['Crossing']=df1['Crossing'].replace({True:1,False:0})
df1['Stop']=df1['Stop'].replace({True:1,False:0})
df1['Traffic_Signal']=df1['Traffic_Signal'].replace({True:1,False:0})

In [39]:
variables=['Distance(mi)','Crossing', 'Stop','Traffic_Signal', 'TimeMinutes',
       'State_Freq', 'Temperature(F)_Escalado_KNN', 'Humidity(%)_Escalado_KNN',
       'Pressure(in)_Escalado_KNN', 'Visibility(mi)_Escalado_KNN',
       'Wind_Speed(mph)_Escalado_KNN']

In [40]:
from sklearn.feature_selection import SelectKBest

In [78]:
sk = SelectKBest(k=3)
sk.fit(df1[variables],df1["Severity"])

SelectKBest(k=3)

In [79]:
sk.scores_

array([35742.17739783, 26103.35510517,  3988.92424193, 44302.85402689,
        1009.0000991 ,  8371.12294845,  2505.18546612,  4011.92565383,
        7464.80465862,   522.88530082,  2105.77300095])

In [80]:
best= [a for a,b in list(zip(variables,sk.get_support())) if b]

In [81]:
best

['Distance(mi)', 'Crossing', 'Traffic_Signal']